# Locomotion




## Play

### 1. Example from repo

Extract two sequences from clips and render in matplotlib.pyplot

In [ ]:
import numpy as np

from mocap.AnimationPlotLines import animation_plot

train_data = np.load('mocap/data/edinburgh_locomotion_train.npz')
test_data = np.load('mocap/data/edinburgh_locomotion_test.npz')

clips = train_data['clips']
clips = np.swapaxes(clips, 1, 2)

index = 0
seq1 = clips[index:index+1]
seq2 = clips[index+1:index+2]

animation_plot([seq1, seq2], interval=15.15)

### 2. Dataset

Dataset

In [ ]:
class MocapDataset(Dataset):
    def __init__(self, path='mocap/data/edinburgh_locomotion_train.npz'):
        self.data = torch.from_numpy(np.load(path)['clips'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        joint_positions = self.data[idx][:, :-3]
        ctrl = self.data[idx][:, -3:]
        return {'j_pos': joint_positions, 'ctrl':ctrl}


### 2. DataLoader

Dataloader

In [ ]:
class MocapDataLoader(DataLoader):
    def __init__(self, dset, batch_size, seq_len=1, overlap_len=0, *args, **kwargs):
        super().__init__(dset, batch_size, *args, **kwargs)
        self.seq_len = seq_len
        self.overlap_len = overlap_len

    def __iter__(self):
        for batch in super().__iter__():
            jpos = batch['j_pos']
            ctrl = batch['ctrl']
            batch_size, n_frames, n_joints = jpos.shape

            reset = True
            for i in range(self.overlap_len, n_frames, self.seq_len):
                j = jpos[:, i, :]
                c = ctrl[:, i, :]
                yield j, c, reset
                reset = False

### 3. Model

Goal - dilated convolutions


1d convolutions

In [ ]:
class Model(nn.Module):
    def __init__(self,
                 in_channels=20,
                 mid_channels=[20, 10],
                 out_size=10):
        super(Model, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, mid_channels[0])
        self.conv2 = nn.Conv1d(mid_channels[0], mid_channels[1])
        self.conv3 = nn.Conv1d(mid_channels[1], mid_channels[2])
        self.out = nn.Linear(mid_channels[2], out_size=10)

    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        return self.out(x)

### Loss Function